# Code 8
- Random Forest
- Hyperparameter Optimization
- GridsearchCV
- Best Features

## 1/ Import Libraries

In [ ]:
#CodeSection1
from google.colab import drive
drive.mount('/mntDrive') 

Mounted at /mntDrive


In [ ]:
#CodeSection2
import pandas as pd
import numpy as np

## 2/ Import Data

In [ ]:
#CodeSection3
train = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/train.csv')
test = pd.read_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/input/test.csv')

## 3/ Skipping Few Steps to simplify

## 4/ Preprocessing Improvement

### 4.1/ Identify Numerical and Categorical Features

In [ ]:
#CodeSection4
# Identify all Numerical and Categorical features
numerical_features = ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
categorical_features = ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area']

### 4.2/ Outlier Strategy

In [ ]:
#CodeSection5

# Write a loop to do the same
for num_var in numerical_features:
  Q1 = train[num_var].quantile(0.25)
  Q3 = train[num_var].quantile(0.75)

  IQR = Q3-Q1

  Lower_Whisker = Q1 - 1.5*IQR
  Upper_Whisker = Q3 + 1.5*IQR

  train[num_var] = train[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)
  test[num_var] = test[num_var].apply(lambda x : Upper_Whisker if x >= Upper_Whisker else x)

### 4.3/ Import Libraries
- Missing Value Imputation - SimpleImputer
- Preprocessing - StandardScaler, OrdinalEncoder
- Pipeline - make_pipeline, make_column_transformer
- Model - Decision Tree

In [ ]:
#CodeSection6

# Import SimpleImputer
from sklearn.impute import SimpleImputer

# Import StandardScaler
from sklearn.preprocessing import StandardScaler, OrdinalEncoder

# Make and Compose Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

# Import Random Forest
from sklearn.ensemble import RandomForestClassifier

### 4.4/ Build Pipeline

In [ ]:
#CodeSection7
# Create Preprocessor Pipeline
preprocessor = make_column_transformer(
    
    (make_pipeline(
    SimpleImputer(strategy = 'median'), 
    StandardScaler()), numerical_features),
    
    (make_pipeline(
    SimpleImputer(strategy = 'most_frequent'),
    OrdinalEncoder(categories = 'auto')), categorical_features),
)

### 4.5/ Divide Data into X and y

In [ ]:
#CodeSection8
X = train.drop(['Loan_Status','Loan_ID'], axis =  1)
y = train['Loan_Status']

### 4.6/ Create Train and Validation Data

In [ ]:
#CodeSection9
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 5) # also see stratify

## 5/ Build Model and Fit

In [ ]:
#CodeSection10
# Create Parameter Grid
parameter = { 'randomforestclassifier__n_estimators' : (23, 24, 25, 26, 27, 28),
             'randomforestclassifier__max_depth' : (2,3,4,5,6,7,8,9,10),
             'randomforestclassifier__criterion' : ('gini', 'entropy'),
             'randomforestclassifier__max_features' : ('auto', 'sqrt', 'log2')
    
}
#(23, 24, 25, 26, 27, 28)
#(2,3,4,5,6,7,8,9,10)
#('gini', 'entropy')
#('auto', 'sqrt', 'log2')

In [ ]:
#CodeSection11
# Create Model Pipeline and Initiate Model
# Change max_depth to find which one gives the best accuracy
model = make_pipeline(preprocessor, RandomForestClassifier())

In [ ]:
#CodeSection12
# Instead of Fit we so Grid Search (also Random Grid Search)
from sklearn.model_selection import GridSearchCV 
model_search = GridSearchCV(model, param_grid = parameter, verbose = True)

In [ ]:
#CodeSection13
# Fit Gridsearch
model_search.fit(X_train,y_train)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1620 out of 1620 | elapsed:  1.8min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('columntransformer',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('pipeline-1',
                                                                         Pipeline(memory=None,
                                                                                  steps=[('simpleimputer',
                                                                                          SimpleImputer(add_indicator=False,
                                                                                                        copy=True,
               

In [ ]:
#CodeSection14
# Get best estimator from the Girdsearch
model_search.best_estimator_.named_steps.randomforestclassifier

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=6, max_features='log2',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=27,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

## 6/ Check Best Features

In [ ]:
#CodeSection15
# Get Feature Importance Score
feat_imp = model_search.best_estimator_.named_steps.randomforestclassifier.feature_importances_
feat_imp

array([0.10913652, 0.0999889 , 0.1237537 , 0.04472901, 0.46364169,
       0.00822858, 0.02142059, 0.03615321, 0.02423887, 0.01178228,
       0.05692665])

In [ ]:
#CodeSection16
# Convert to Series with Feature Names
imp_feat = pd.Series(feat_imp,index=X_train.columns.tolist())

In [ ]:
#CodeSection17
# List Feature Importance
imp_feat.sort_values(ascending=False) # You also plot the same

Self_Employed        0.463642
Dependents           0.123754
Gender               0.109137
Married              0.099989
Property_Area        0.056927
Education            0.044729
LoanAmount           0.036153
Loan_Amount_Term     0.024239
CoapplicantIncome    0.021421
Credit_History       0.011782
ApplicantIncome      0.008229
dtype: float64

## 6/ Check Accuracy of Model on Train Data

In [ ]:
#CodeSection18
# Predict on Train Data
y_train_pred = model_search.predict(X_train)
y_val_pred = model_search.predict(X_val)

### We can use a Accuracy Function from Metrics
- Check Train Accuracy

In [ ]:
#CodeSection19
# Import metrics library
from sklearn.metrics import accuracy_score

In [ ]:
#CodeSection20
# Print Train Accuracy
print(f" Train Accuracy : {accuracy_score(y_train, y_train_pred):0.1%}")
print(f" Validation Accuracy : {accuracy_score(y_val, y_val_pred):0.1%}")

 Train Accuracy : 84.5%
 Validation Accuracy : 79.7%


## 7/ Predict and Submission

### Predict on "Test Data"

In [ ]:
#CodeSection21
# Get all the X Variables from the Test Dataset
X_test = test.drop(['Loan_ID'], axis =  1)

# Predict on X_test Data ("X_test_prep")
X_test_prep = model_search.predict(X_test)

### Create Submission File

In [ ]:
#CodeSection22
submission = pd.DataFrame({
    'Loan_ID' : test['Loan_ID'],
    'Loan_Status' : X_test_prep
})

### Export Submission File

In [ ]:
#CodeSection23
submission.to_csv('/mntDrive/MyDrive/Project MasterMind/1 - Planning Stage/2- Research/6- New Teaching Method/Loan Prediction (Webinar Flow)/Solution Codes/output/O8_RF_Best_Features.csv', index = False)

In [ ]:
# LB Accuracy : 0.7777